In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.model_selection import train_test_split # dataset split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error,mean_absolute_error # 평가도구
from sklearn.metrics import confusion_matrix, accuracy_score # model 평가
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기
data = pd.read_csv(r'/광진구_최종.csv', encoding='CP949')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 63 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   기준_년분기_코드        1539 non-null   int64  
 1   행정동_코드           1539 non-null   int64  
 2   행정동_코드_명         1539 non-null   object 
 3   서비스_업종_코드        1539 non-null   object 
 4   서비스_업종_코드_명      1539 non-null   object 
 5   당월_매출_금액         1539 non-null   int64  
 6   당월_매출_건수         1539 non-null   int64  
 7   주중_매출_금액         1539 non-null   int64  
 8   주말_매출_금액         1539 non-null   int64  
 9   월요일_매출_금액        1539 non-null   int64  
 10  화요일_매출_금액        1539 non-null   int64  
 11  수요일_매출_금액        1539 non-null   int64  
 12  목요일_매출_금액        1539 non-null   int64  
 13  금요일_매출_금액        1539 non-null   int64  
 14  토요일_매출_금액        1539 non-null   int64  
 15  일요일_매출_금액        1539 non-null   int64  
 16  시간대_00~06_매출_금액  1539 non-null   int64  
 17  시간대_06~11_매출_금

In [3]:
# feature, target 분리 ('폐업_점포_수' 삭제)
feature = data.drop(columns=['기준_년분기_코드','행정동_코드','행정동_코드_명','서비스_업종_코드','서비스_업종_코드_명','폐업_률','주중_매출_금액','주말_매출_금액','주중_매출_건수','주말_매출_건수','1년 생존율','3년 생존율','5년 생존율','점포_수','폐업_점포_수'],axis=1)
target = data['폐업_률']


# train, test 분리
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.3, random_state = 2024)

# SimpleImputer를 사용하여 결측치 대체
imputer = SimpleImputer(strategy='mean')  # 평균값으로 대체
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

std_scaler = StandardScaler()
X_std_scaled = std_scaler.fit_transform(X_train)

X_std_scaled_test = std_scaler.fit_transform(X_test)


print(X_std_scaled)


[[-0.15913516  0.56069818 -0.08065282 ...  0.42729534  0.48906248
  -0.18557655]
 [-0.31009881 -0.40867162 -0.30771753 ... -0.09585405 -0.23346192
   0.58373039]
 [ 1.18954901  1.38413577  1.38319178 ...  0.07852908  1.93411129
   2.27620566]
 ...
 [-0.45131023 -0.58675165 -0.4648189  ... -0.79338656 -0.59472413
  -0.95488349]
 [-0.49321952 -0.61281744 -0.4945843  ...  0.77606159 -0.23346192
  -0.95488349]
 [ 0.55042561  2.97161474  0.61064822 ... -0.27023718  0.12780028
   1.50689872]]


In [4]:
from sklearn.model_selection import GridSearchCV

# alpha 값 후보 리스트
alphas = np.logspace(-4, 4, 10)

# GridSearchCV를 사용하여 최적의 alpha 값 찾기
lasso_cv = Lasso()
param_grid = {'alpha': alphas}
grid_search = GridSearchCV(lasso_cv, param_grid, cv=5)
grid_search.fit(X_std_scaled, Y_train)

# 최적의 alpha 값
best_alpha = grid_search.best_params_['alpha']
print("최적의 alpha:", best_alpha)

# 최적의 alpha 값으로 라쏘 모델 구축
lasso = Lasso(alpha=best_alpha, max_iter=1000).fit(X_std_scaled, Y_train)

# 회귀계수가 0이 아닌 특성의 인덱스를 찾음
non_zero_coef_indices = [i for i, coef in enumerate(lasso.coef_) if coef != 0]

# 인덱스를 사용하여 원래 feature의 이름을 가져옴
feature_list = feature.columns[non_zero_coef_indices]

print("선택된 특성:", feature_list)
print(lasso.coef_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.037e+04, tolerance: 2.455e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.025e+04, tolerance: 2.451e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.103e+04, tolerance: 2.676e

최적의 alpha: 0.3593813663804626
선택된 특성: Index(['개업_율'], dtype='object')
[-0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.          0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.          0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.79127375 -0.          0.        ]


In [5]:
# model 생성 : train set
lr = linear_model.Lasso(alpha = best_alpha)
lr.fit(X_std_scaled, Y_train)

lr.coef_

# 회귀계수가 0이 아닌 특성의 인덱스를 찾음
non_zero_coef_indices = [i for i, coef in enumerate(lr.coef_) if coef != 0]

# 인덱스를 사용하여 원래 feature의 이름을 가져옴
feature_list = feature.columns[non_zero_coef_indices]

print("선택된 특성:", feature_list)
print(lr.coef_)

선택된 특성: Index(['개업_율'], dtype='object')
[-0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.          0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
  0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.          0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.79127375 -0.          0.        ]


In [6]:
# train rmse
#train_predict = lr.predict(X_train_imputed)
#rmse1=print("RMSE':{}".format(math.sqrt(mean_squared_error(train_predict, Y_train))) )

# test rmse
test_predict = lr.predict(X_std_scaled_test)
rmse2=print("test RMSE:{}".format(math.sqrt(mean_squared_error(test_predict, Y_test))) )

test RMSE:5.96584808077864


In [7]:
# train mae
#train_predict = lr.predict(X_train_imputed)
#rmse1=print("MAE':{}".format((mean_absolute_error(train_predict, Y_train))) )

# test mae
test_predict = lr.predict(X_std_scaled_test)
rmse2=print("test MAE:{}".format((mean_absolute_error(test_predict, Y_test))) )

test MAE:4.1190893579454855
